In [3]:
import numpy as np
import pandas as pd
idx = pd.IndexSlice
import xarray as xr

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from itertools import product

# import sys
# sys.path.append('../funcs')
# from taus import decorrelation_temporal_model, fit_coh_decay_model

In [83]:
import matplotlib
class FixPointNormalize(matplotlib.colors.Normalize):
    """ 
    Inspired by https://stackoverflow.com/questions/20144529/shifted-colorbar-matplotlib
    Subclassing Normalize to obtain a colormap with a fixpoint 
    somewhere in the middle of the colormap.

    This may be useful for a `terrain` map, to set the "sea level" 
    to a color in the blue/turquise range. 
    """
    def __init__(self, vmin=None, vmax=None, sealevel=0, col_val = 0.21875, clip=False):
        # sealevel is the fix point of the colormap (in data units)
        self.sealevel = sealevel
        # col_val is the color value in the range [0,1] that should represent the sealevel.
        self.col_val = col_val
        matplotlib.colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.sealevel, self.vmax], [0, self.col_val, 1]
        return np.ma.masked_array(np.interp(value, x, y))

colors_undersea = plt.cm.terrain(np.linspace(0, 0.17, 56))
colors_land = plt.cm.terrain(np.linspace(0.25, 1, 200))
# combine them and build a new colormap
colors = np.vstack((colors_undersea, colors_land))
cut_terrain_map = matplotlib.colors.LinearSegmentedColormap.from_list('cut_terrain', colors)

In [8]:
dss = {fp.stem: xr.open_dataset(fp) for fp in sorted(list(Path('/Users/rdcrlzh1/Documents/uavsar-coherence/uavsar').glob('*.nc')))}

home_dir = Path('/Users/rdcrlzh1/Documents/uavsar-coherence/')
fig_dir = home_dir.joinpath('figures', 'examples')

In [89]:
for stem, ds_full in dss.items():
    for heading in ds_full.heading.values:
        # if stem != 'stlake_full': continue
        fig, axes = plt.subplots(2, 3, figsize = (16,8))
        axr = axes.ravel()
        ds = ds_full.sel(pol = 'HH', heading = heading)
        from itertools import product
        n = 0
        for t1, t2 in product(ds.time1, ds.time2):
            if (~ds['cor'].sel(time1 = t1, time2 = t2).isnull()).sum() == 0: continue
            n += 1

        ds['cor'].mean(dim = ['time1', 'time2']).dropna('y', how = 'all').plot(ax = axr[0], cbar_kwargs={'label':'Mean Coherence []'}, cmap = 'Greys_r')
        bds = ds['cor'].mean(dim = ['time1', 'time2']).dropna('y', how = 'all')
        axr[0].set_title(f'Mean Coherence\nn = {n}')

        ds['cor'].std(dim = ['time1', 'time2']).dropna('y', how = 'all').plot(ax = axr[1], cbar_kwargs={'label':'Coherence Std []'})
        axr[1].set_title(f'Std Coherence\nn = {n}')

        np.rad2deg(ds['inc']).dropna('y', how = 'all').plot(ax = axr[2], vmax = 90, cbar_kwargs={'label':'Incidence Angle [°]'}, cmap = 'magma')
        axr[2].set_title('Incidence Angle')

        ds['tree_perc'].where(~bds.isnull()).dropna('y', how = 'all').plot(ax = axr[3], cbar_kwargs={'label':'Tree Percentage [%]'}, cmap = 'Greens')
        axr[3].set_title('Tree Percentage')

        norm = FixPointNormalize(sealevel=ds['dem'].min(), vmax=ds['dem'].max())
        ds['dem'].where(~bds.isnull()).dropna('y', how = 'all').plot(ax = axr[4], cbar_kwargs={'label':'Elevation [m]'}, norm=norm, cmap=plt.cm.terrain)
        axr[4].set_title('DEM')

        ds['cor'].plot.hist(bins = 50, ax= axr[5], density = False)
        axr[5].set_title('All Pixel Coherence Histogram')

        for ax in axes.ravel()[:-1]:
            ax.set_xlabel('')
            ax.set_ylabel('')
            ax.xaxis.set_major_locator(plt.MaxNLocator(3))
            ax.yaxis.set_major_locator(plt.MaxNLocator(3))

        for ax in axes[:, 0]:
            ax.set_ylabel('Latitude [°]')
        for ax in axes[-1, :]:
            ax.set_xlabel('Longitude [°]')
        
        for ax in axes[:, 1].ravel(): ax.set_yticklabels([])
        axes[0, 2].set_yticklabels([])
        for ax in axes[0, :-1].ravel(): ax.set_xticklabels([])

        axes[1, 2].ticklabel_format(axis = 'y', style = 'sci', scilimits=(0,0))


        plt.tight_layout()
        fig.savefig(fig_dir.joinpath(stem.replace('_full', f'_{heading}') + '_ex.png'))
        plt.close(fig)


/Users/rdcrlzh1/miniforge3/envs/coherence/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/rdcrlzh1/miniforge3/envs/coherence/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/rdcrlzh1/miniforge3/envs/coherence/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/rdcrlzh1/miniforge3/envs/coherence/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/rdcrlzh1/miniforge3/envs/coherence/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedo